tf.random.normal(
    shape,
    mean=0.0,
    stddev=1.0,
    dtype=tf.dtypes.float32,
    seed=None,
    name=None
) -> Outputs random values from a normal distribution

tf.nn.softmax(
    logits,
    axis=None,
    name=None,
    dim=None
)

tf.math.reduce_sum(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
) Computes the sum of elements across dimensions of a tensor. 
Reduces input_tensor along the dimensions given in axis

tf.math.reduce_mean(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
) Computes the mean of elements across dimensions of a tensor.
Reduces input_tensor along the dimensions given in axis

In [32]:
import tensorflow as tf
import numpy as np
import json
import random

In [45]:
###
### Citest json files si creeaza lista de cuvinte unice din venues. Scrie rezultatul in fisier.
### return numarul de cuvinte.
###
def creeaza_lista_cuvinte_unice():
    lista_unica_cuvinte = []
    
    ## Citeste lista de venues
    r = open('../Date/Initiale/lista_venue_completa.txt', 'r')
    lista_venues = json.load(r)
    r.close()
    
    ## Creeaza lista de cuvinte unice din venues
    for venue in lista_venues:
        lista_unica_cuvinte += venue
    lista_unica_cuvinte = set(lista_unica_cuvinte)
    
    w = open('../Date/Initiale/lista_unica_cuvinte_venues.txt','w')
    w.write(json.dumps(list(lista_unica_cuvinte)))
    w.close()

    return len(lista_unica_cuvinte)

###
### Citest json files si creeaza lista de cuvinte unice din abstract. Scrie rezultatul in fisier.
### return numarul de cuvinte.
###
def creeaza_lista_cuvinte_unice_abstract():
    lista_unica_cuvinte = []
    
    ## Citeste lista de venues
    lista_abstr = []
    for i in range(0,169):
        file_ = '../Date/Initiale/Complet/file' + str(i) + '.txt'
        r = open(file_, 'r')
        for line in r:
            crt_art = json.loads(line)
            lista_abstr.append(crt_art['abstract'])
            
        r.close()
    
    w = open ('../Date/Initiale/lista_abstr.txt','w')
    w.write(json.dumps(lista_abstr))
    
    ## Creeaza lista de cuvinte unice din abstract
    for abstr in lista_abstr:
        lista_unica_cuvinte += abstr
    lista_unica_cuvinte = set(lista_unica_cuvinte)
    
    w = open('../Date/Initiale/lista_unica_cuvinte_abstr.txt','w')
    w.write(json.dumps(list(lista_unica_cuvinte)))
    w.close()

    return len(lista_unica_cuvinte)

###
### Citeste lista de cuvinte unice si creeaza dictionar {cuvant:id}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_word2int(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    word2int = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        word2int[cuvant] = i
        
    w = open(out_file,'w')
    w.write(json.dumps(word2int))
    w.close()
    
    
###
### Citeste lista de cuvinte unice si creeaza dictionar {id:cuvant}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_int2word(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    int2word = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        int2word[i] = cuvant
        
    w = open(out_file,'w')
    w.write(json.dumps(int2word))
    w.close()

###
### Creeaza un vector de tip one hot.
### param index: pozitia din vector pe care se va pune 1.
### param len_vect: lungimea vectorului.
### return hot_vect: vectorul rezultat.
###
def one_hot(index, len_vect):
    hot_vect = np.zeros(len_vect)
    hot_vect[index] = 1
    return hot_vect

###
###
### param word2int_file: calea spre fisierul de word2int.
### param docs_file: lista cu toate venues.
### param DIM_FEREASTRA: dimensiunea ferestrei considerate.
### param DIM_VOCABULAR: dimensiunea vocabularului.
### returneaza X_train si Y_train
def creeaza_lista_vecini(word2int_file, docs_file, DIM_FEREASTRA, DIM_VOCABULAR):
    # Citeste word2int
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    # Citeste lista documente
    r = open(docs_file, 'r')
    lista_doc = json.load(r)
    r.close()
    
    X_train = [] 
    Y_train = [] 
    
    for doc in lista_doc:
        for word_index, word in enumerate(doc):
            for vecin in doc[max(word_index - DIM_FEREASTRA, 0) : min(word_index + DIM_FEREASTRA, len(doc)) + 1] : 
                if vecin != word:
                    X_train.append(one_hot(word2int[ word ], DIM_VOCABULAR))
                    Y_train.append(one_hot(word2int[ vecin ], DIM_VOCABULAR))

    X_train = np.asarray(X_train)
    Y_train = np.asarray(Y_train)
    
    return X_train, Y_train

def log(word2int_file, vectors, file):
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    dict_ = {}
    for elem in word2int:
        dict_[elem] = vectors[word2int[elem]].tolist()
    w = open (file, 'w')
    w.write(json.dumps(dict_))
    w.close()       
    

def train ():
    DIM_VOCABULAR = creeaza_lista_cuvinte_unice_abstract()
    #creeaza_word2int('../Date/Initiale/lista_unica_cuvinte_venues.txt','../Date/Initiale/word2int_venues.txt')
    #creeaza_int2word('../Date/Initiale/lista_unica_cuvinte_venues.txt','../Date/Initiale/int2word.txt')
    
    DIM_FEREASTRA = 2
    X_train, Y_train = creeaza_lista_vecini('../Date/Initiale/word2int_venues.txt', '../Date/Initiale/lista_venue_completa.txt', DIM_FEREASTRA, DIM_VOCABULAR)

    #### Creare retea
    X = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))
    Y_eticheta = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))

    ## Din layer de input in hidden layer
    EMBEDDING_DIM = 50
    W1 = tf.Variable(tf.random_normal([DIM_VOCABULAR, EMBEDDING_DIM]))
    b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
    layer_hidden = tf.add(tf.matmul(X,W1), b1)

    ## Din hidden layer in output layer
    W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, DIM_VOCABULAR]))
    b2 = tf.Variable(tf.random_normal([DIM_VOCABULAR]))
    output_layer = tf.nn.softmax(tf.add( tf.matmul(layer_hidden, W2), b2))
    
    #### Instantiere model tensorflow
    with tf.device('/gpu:0'):
        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)

        # Functia de loss -> cross entropy
        cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(Y_eticheta * tf.log(tf.clip_by_value(output_layer,1e-10,1.0)), reduction_indices=[1]))
        #is actually a horrible way of computing the cross-entropy. In some samples, certain classes could be excluded with certainty after a while, resulting in y_conv=0 for that sample. That's normally not a problem since you're not interested in those, but in the way cross_entropy is written there, it yields 0*log(0) for that particular sample/class. Hence the NaN.
        
        # compute_gradients + apply_gradients = minimize
        learning_rate = 0.02
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)

        loss_stochastic = []
        
        i = 0
        while(1):

            for k in range(DIM_VOCABULAR):
                if i > 10000 and i % 100 == 0:
                    vectors = sess.run(W1 + b1)
                    f_log = '../Models/modele_tensorflow/model' + str(i) + '.txt'
                    log('../Date/Initiale/word2int_venues.txt', vectors, f_log )

                
                i += 1
                rand = random.randint(0,DIM_VOCABULAR)
                x_rand = X_train[rand].reshape(1,1416) # Transpose to the correct shape
                y_rand = Y_train[rand].reshape(1,1416)

                sess.run(train_step, feed_dict={X: x_rand, Y_eticheta:y_rand})
                
                temp_loss = sess.run(cross_entropy_loss, feed_dict={X: x_rand, Y_eticheta:y_rand})
                loss_stochastic.append(temp_loss)
                
                w = open('../Models/modele_tensorflow/loss.txt','a')
                w.write(str(temp_loss))
                w.write(' ')
                w.close()
                if k % 100 == 0:
                    print('Loss = ', temp_loss, i)
                    w = open('../Models/modele_tensorflow/lossk.txt','a')
                    w.write(str(temp_loss))
                    w.write(' ')
                    w.close()


In [46]:
vectors = train()

In [ ]:
loss is :  5.676243 18087